# Choosing Wisely Metrics
(ICD9 and ICD10 compatible) <br>
This algorithm was developed by Colin Walsh's group at Vanderbilt University Medical Center to implement 11 Choosing Wisely metrics using criteria defined by Colla et al. (JGIM 2014) on ICD-10 based patient data. 

This notebook has the following parts:
1) Set-up environment <br>
2) Load reference data, with preprocessing <br>
3) Load patient data from SQL server, with preprocessing <br>
4) Load General Equivalence Mappings, with preprocessing <br>
5) Instantiate 11 classes, one for each of the 11 metrics <br>
6) Calculate the each metric's prevalence, using different mapping/harmonization methods <br>
7) Consoliate and export results <br>

[Last edited Dec 15, 2018, J Angiolillo]

In [ ]:
import os
os.getcwd()

In [ ]:
# import most basic tools
from cw_package.setup_cw_env import *
from pylab import *
import pickle, sys, re
jacks_verification

In [ ]:
"""
This block of code preps a report on a monthly basis, as specified
and saves it as named to the location identified below.
"""
reportStart='2014-03-31' 
claimsStart='2015-03-31'
reportOnMonths=['2016_4','2016_5','2016_6','2016_7','2016_8','2016_9']
demoStartDate='2016-04-01'
demoEndDate='2016-10-01'
admsEndDate='2016-12-01'
coded_date = '2017_07_12allfeedwith_I10' # export path name

In [ ]:
# build the directories for exporting results
try:
    os.mkdir('exported/'+coded_date)
    os.mkdir('exported/Final Charts/'+coded_date)
except:
    pass

In [ ]:
#  # this was abandoned, and non functional
# def incrementmonth(str_x):
#     if str_x[-1]=='9':
#         str_x=str_x[:-2]+'10'
#     elif str_x[-2:]=='12':
#         year=str_x[:-3]
#         year=str(int(year)+1)
#         str_x=year+'-'+'01'        
#     else:
#         str_x=str_x[:-1]+str(int(str_x[-1])+1)
#     return str_x

# m1=demoStartDate[:7]
# m2=incrementmonth(m1)
# m3=incrementmonth(m2)
# m4=incrementmonth(m3)
# m5=incrementmonth(m4)
# m6=incrementmonth(m5)
# enddt=incrementmonth(m6)

# print(enddt)

### Import needed modules

In [ ]:
# import simple utility modules
from cw_package import add_age, add_month, timeBtwnDex, calculate_age, downloadDemographicsByDate, subsetByDemographics#, output_monthly
from cw_package import prDF
from cw_package import cwsql_codeviewer
from cw_package import df_claimtrimmer

In [ ]:
## Import main functions
#    preprocessing and completing general equivalence mappings
from cw_package import createuniquemap, simplify_xgem, seeambiguousmaps
#    preprocessing claims data
from cw_package import preprocessingclaims
#    methods for parsing reference tables and applying them to patient data
from cw_package import keepindications_removeexclusions, vect_mrn
#    Class definition for the metric
from cw_package import MetricClass
#    debugging method for inspecting single metric in single GEM implementation --- requires editing code as needed
from cw_package import testallwisely

### Load in Reference Criteria, on ICD9 basis

In [ ]:
"""
This block loads in the reference arrays/dataframes adapted from Colla et al's JGIM paper.
which will be used for parsing parse claims and service data
"""
#This sheet includes the lookups for the context of services/procedures being questioned by CW (ie "low back pain"):
            ## NOTE: indications ref-table is altered by Jack - surgical proc removed
indications=pd.read_excel('./ref/IndicationsLookup_wo_op_20170211a.xlsx',header=0) 
#This sheet includes valid reasons for service, not wasteful:
#redflags=pd.read_excel('./ref/RedFlagsLookup_20170211a.xlsx',header=0) ### this is the old original file
redflags=pd.read_csv('./ref/Redflags_c_EandPcodes_20170322.csv')
#This sheet includes services that fall in bin of the CW-service
services=pd.read_excel('./ref/ServiceLookup_20170420.xlsx',header=0) 
#This sheet gives parameters of patients searched over
democriteria=pd.read_excel('./ref/DemoLookup_20160630.xlsx',header=0) 
#This is specific to preop CW; selecting all the CPT codes that fall within legacy BETOS codes
preopref=pd.read_csv('./ref/SurgeryBETOSCPTLookup_20160329.csv',header=0) 

Preprocess ICD-9 Reference Criteria

In [ ]:
# Preprocessing reference data: Typing and Cleaning

#services.code=services.code.apply(lambda x:str(x).lower()) ## services codes should not be forced to lower case
services['code']=services['code'].apply(lambda x: str(x).replace('.',''))
#
redflags['subcode']=redflags['code'].apply(lambda x: str(x).replace('.',''))
indications['subcode']=indications['code'].apply(lambda x: str(x).replace('.',''))
# Deal with StartsWith Problem
# Create a startWith column
# Populate it with "1's" if subcode includes an x
# Now remove Xs
services['startWith']=0
services['code']=services['code'].apply(lambda x: str(x).replace('X','x'))
services['startWith'].loc[services['code'].str[-1]=='x']=1
services['code']=services['code'].apply(lambda x: str(x).replace('x',''))

redflags['startWith']=0
redflags['subcode']=redflags['subcode'].apply(lambda x: str(x).replace('X','x'))
redflags['startWith'].loc[redflags['subcode'].str[-1]=='x']=1
redflags['subcode']=redflags['subcode'].apply(lambda x: str(x).replace('x',''))

indications['startWith']=0
indications['subcode']=indications['subcode'].apply(lambda x: str(x).replace('X','x'))
indications['startWith'].loc[indications['subcode'].str[-1]=='x']=1
indications['subcode']=indications['subcode'].apply(lambda x: str(x).replace('x',''))
print('done.')

In [ ]:
#Checkpoint
len(indications),len(redflags),len(services),len(democriteria)
print('completed.')

Load ICD-10 reference data

In [ ]:
# Load the Novel ICD-10 to ICD-10 reference tables developed by Harbor Lab/Colin Walsh
# sheet names here specify the final tables implemented from the file.
indic_10 = pd.read_excel('./ref/ICD10_basis/I10_New_reftables2017_04_25.xlsx', sheetname='indications_w_ICD10')
#red_10   =  pd.read_excel('./ref/ICD10_basis/I10_New_reftables2017_04_25.xlsx', sheetname='redflags_ICD10_wHCCCCS_Apr20') # this was old redflags table, not-used for final data
red_10   =  pd.read_excel('./ref/ICD10_basis/I10_New_reftables2017_04_25.xlsx', sheetname='redflags_ICD10_wo_I10htn')
serv_10  = pd.read_excel('./ref/ICD10_basis/I10_New_reftables2017_04_25.xlsx', sheetname='services_w_ICD10_wo_dxcodes')

In [ ]:
# get rid of extra,hidden preprocessing
red_10=red_10[['category','class','old_code','code','key','label']]

In [ ]:
# Preprocess novel ICD-10 reference data
indic_10['subcode']=indic_10['code'].apply(lambda x: str(x).replace('.',''))
indic_10['startWith']=0
indic_10['startWith'].loc[indic_10['subcode'].str.contains('_',case=False)]=1
indic_10['subcode']=indic_10['subcode'].apply(lambda x: str(x).replace('_',''))
indic_10=indic_10[['label','code','class','key','subcode','startWith']]

red_10['subcode']=red_10['code'].apply(lambda x: str(x).replace('.',''))
red_10['startWith']=0
red_10['startWith'].loc[red_10['subcode'].str.contains('_',case=False)]=1
red_10['subcode']=red_10['subcode'].apply(lambda x: str(x).replace('_',''))

serv_10.code = serv_10.code.apply(lambda x: str(x).replace('.',''))
serv_10['startWith']=0
serv_10['startWith'].loc[serv_10['code'].str.contains('_',case=False)]=1
serv_10['code'] = serv_10['code'].apply(lambda x: str(x).replace('_',''))

In [ ]:
# restrict columns
red_10=red_10[['label','category','old_code','code','class','key','subcode','startWith']]

Get rid of redundant subcodes/stems within each metric, keep at least one of them

In [ ]:
red_10.drop_duplicates(subset = ['class','key','subcode','startWith'], keep='first', inplace=True)
indic_10.drop_duplicates(subset=['class','key','subcode','startWith'], keep='first', inplace=True)

### Load and preprocess other reference tables (ICD9 descriptions, NHAMCSref, HCC, CCS)

In [ ]:
#os.chdir('..')

In [ ]:
# load terminologies' labels
icdref=pd.read_pickle('././pickle/icdref.p')
nhamcsref=pd.read_table('./ref/DrugCodeLookupTable.txt',header=0)

In [ ]:
# load more terminologies' maps (ICD to HCC)
##ICD9 HCC Codes
""" 
    HCC codes are maintained by CMS, there are more than 200 (only ~79 used in risk adjustment)
"""
hccref={} 
# following lines load the HCC references for ICD-9 and ICD-10 into the reference dictionary
hccref['9']=pd.read_pickle('./pickle/hccref.p')
hccref['9'].rename(columns={'2013 CMS-HCC Model Category':'hcclev'},inplace=True)
##ICD10 HCC Codes
hccref['10']=pd.read_csv('./ref/hccCodesFormatted2016_ICD10.csv',header=0)
hccref['10'].rename(columns={'Diagnosis Code':'subcode','CMS-HCC PACE/ESRD Model Category V21':'hcclev'},inplace=True)


In [ ]:
# more HCC reference maps
# to capture the history of remote cancer
jackhcc9 = pd.read_excel('./ref/additional hcc codes.xlsx', sheetname='ICD9')
jackhcc10 = pd.read_excel('./ref/additional hcc codes.xlsx', sheetname='ICD10')

jackhcc9['subcode'] = jackhcc9['code'].apply(lambda x: str(x).replace('.',''))
jackhcc9['hcclev']=   jackhcc9['hcclev'].apply(lambda x: str(x))
jackhcc10['subcode'] = jackhcc10['code'].apply(lambda x: str(x).replace('.',''))
jackhcc10['hcclev']=   jackhcc10['hcclev'].apply(lambda x: str(x))

jackhcc9.drop('code', axis=1, inplace=True)
jackhcc10.drop('code', axis=1,inplace=True)

In [ ]:
hccref['9']= pd.concat([hccref['9'],jackhcc9])
hccref['10']=pd.concat([hccref['10'],jackhcc10])

In [ ]:
# load CCS reference maps (ICD to CCS)
""" 
    CCS codes are maintained by AHRQ
"""
ccsref={}
ccsref['9']=pd.read_csv('./ref/ccsCodesFormatted2015.csv',header=0)
#print(type(ccsref['9']))
ccsref['9']=ccsref['9'].apply(lambda x:x.str.replace("'",''))
ccsref['9'].rename(columns={"'ICD-9-CM CODE'":'subcode',"'CCS CATEGORY'":'ccslev'},inplace=True)
ccsref['10']=pd.read_csv('./ref/ccsCodesFormatted2016_ICD10.csv',header=0)
ccsref['10']=ccsref['10'].apply(lambda x:x.str.replace("'",''))
ccsref['10'].rename(columns={"'ICD-10-CM CODE'":'subcode',"'CCS CATEGORY'":'ccslev'},inplace=True)
# following two lines of code remove extraneous spaces from integers
ccsref['9']['ccslev']=ccsref['9']['ccslev'].apply(lambda x:int(x.replace(" ",'')))
ccsref['10']['ccslev']=ccsref['10']['ccslev'].apply(lambda x:int(x.replace(" ",'')))

Remove any trailing spaces from 'subcode' strings

In [ ]:
hccref['9']['subcode']=hccref['9']['subcode'].apply(lambda x: str(x).replace(' ',''))
hccref['10']['subcode']=hccref['10']['subcode'].apply(lambda x: str(x).replace(' ',''))
ccsref['9']['subcode']=ccsref['9']['subcode'].apply(lambda x:str(x.replace(" ",'')))
ccsref['10']['subcode']=ccsref['10']['subcode'].apply(lambda x:str(x.replace(" ",'')))


Get rid of the decimals in the HCC/CCS codes

In [ ]:
###################LATE EDIT#######
ccsref['9']['ccslev']=ccsref['9']['ccslev'].apply(lambda x: str(x).partition(".")[0])
ccsref['10']['ccslev']=ccsref['10']['ccslev'].apply(lambda x:str(x).partition(".")[0])


In [ ]:
hccref['9']['hcclev']=hccref['9']['hcclev'].apply(lambda x: str(x).partition(".")[0])
hccref['10']['hcclev']=hccref['10']['hcclev'].apply(lambda x: str(x).partition(".")[0])

In [ ]:
ccsref['9'].loc[ccsref['9']['ccslev']=='nan','ccslev']=None
ccsref['10'].loc[ccsref['10']['ccslev']=='nan','ccslev']=None
hccref['9'].loc[hccref['9']['hcclev']=='nan','hcclev']=None
hccref['10'].loc[hccref['10']['hcclev']=='nan','hcclev']=None

In [ ]:
hcc_9=hccref['9'].rename(columns={'hcclev':'hcclev9'}, inplace=False)
ccs_9=ccsref['9'].rename(columns={'ccslev':'ccslev9'}, inplace=False)
hcc_10=hccref['10'].rename(columns={'hcclev':'hcclev10'}, inplace=False)
ccs_10=ccsref['10'].rename(columns={'ccslev':'ccslev10'}, inplace=False)

### Load Institution's Protected Health Info tables from SQL into pandas dataframes 

In [ ]:
# For reference, to see the most recent recorded SQL code used to build SQL tables, use following [hashed] command
#cwsql_codeviewer()

Claims data (PHI) - load and preprocess to conform with reference table formatting

In [ ]:
db=DB()
icd=pd.read_sql("SELECT * FROM J_cw_icd WHERE EVT_DATE>='"+claimsStart+"' AND EVT_DATE<'"+demoEndDate+"';",db)

In [ ]:
icd['ICD10_subcode']=icd['CODE'].str.replace('.','')
print('okay, done.')

In [ ]:
icd['ICD9_subcode']=icd['ICD9_CODE'].str.replace('.','')
print('okay, done.')

In [ ]:
icd=icd.merge(hccref['10'][['subcode','hcclev']],left_on='ICD10_subcode',right_on='subcode',how='left')
print('okay done again')

In [ ]:
icd=icd.merge(ccsref['10'][['subcode','ccslev']],left_on='ICD10_subcode',right_on='subcode',how='left')
icd['AgeAtTest']=icd['AGEATCLAIM']/365.25
#prDF(icd)
len(icd) 
print('done.')

Service data (PHI) - load and preprocess to conform with reference table formatting

In [ ]:
db=DB()
cpt=pd.read_sql("SELECT * FROM J_cw_cpt WHERE EVT_DATE>='"+reportStart+"' AND EVT_DATE<'"+demoEndDate+"';",db)

In [ ]:
cpt['AgeAtTest']=cpt['AGEATCLAIM']/365.25
#drop the CPT's without ENC_ID
cpt=cpt[cpt['ENC_ID']!='']
#prDF(cpt)
len(cpt) # this line is for code-quality control. Checking to see if the data is matching up.
print('done.')

Individual patient demographics (PHI) - load and preprocess to conform with reference table formatting

In [ ]:
db=DB()
demo=pd.read_sql("SELECT * FROM V_PATIENT_MASTER WHERE MRN IN (SELECT DISTINCT(MRN) FROM V_ENC_MP WHERE ADM_DT BETWEEN '2000-01-01' AND '"+demoEndDate+"');",db)
demo['DOB']=pd.to_datetime(demo['DOB'],infer_datetime_format=True,errors='ignore')
demo['current_age']=demo['DOB'].apply(calculate_age)
#prDF(demo)
len(demo)
print('done.')

Dexa scan data (PHI) - load and preprocess to conform with reference table formatting

In [ ]:
db=DB()
dexa_pd=pd.read_sql("select MRN, ENC_ID, EVT_DATE, CODE from J_CW_DEXA WHERE EVT_DATE>='"+reportStart+"' AND EVT_DATE<'"+demoEndDate+"';",db)
dexa_pd=add_age(dexa_pd,demo)
len(dexa_pd)
print('done.')

Drug Rx data (PHI) - load and preprocess to conform with reference table formatting

In [ ]:
db=DB()
drug=pd.read_sql("SELECT * FROM J_CW_RX WHERE ENTRY_DATE>='"+reportStart+"' AND ENTRY_DATE<'"+demoEndDate+"'",db)
drug['AgeAtTest']=drug['AGEATCLAIM']/365.25
len(drug)
print('done.')

Admissions data (PHI) - load and preprocess to conform with reference table formatting

In [ ]:
#  Get admissions data
db=DB()
adms_pd=pd.read_sql("SELECT MRN as ADM_MRN, ENC_ID as ADM_ENC_ID, ADM_DT, DIS_DT from J_CW_ADMS where ADM_DT >= '"+reportStart+"' AND ADM_DT<='"+admsEndDate+"';",db)
#prDF(adms_pd)
#len(adms_pd)
print('done.')

Further preprocess PHI data

In [ ]:
#Force lowercase
drug.MED_NAME=drug.MED_NAME.apply(lambda x:str(x).lower())
nhamcsref['Drug Names']=nhamcsref['Drug Names'].apply(lambda x:str(x).lower())
drug['FIRST_NAME']=drug.MED_NAME.apply(lambda x:str(x).split(' ',1)[0])
drug=drug.merge(nhamcsref,left_on='FIRST_NAME',right_on='Drug Names',how='left')
# standardize drug's column names to match those important ones of the cpt table
drug.rename(columns={'ENTRY_DATE':'EVT_DATE','Drug Codes':'CODE'},inplace=True)
#################################################
drug.loc[drug.MED_NAME=='oxycodone-acetaminophen','CODE']= 'd03431'
drug.loc[drug.MED_NAME=='hydrocodone-acetaminophen','CODE']= 'd03428'
#################################################
drug=drug[drug.CODE.notnull()]

### Trying to reduce size of PHI data for leaner memory use

In [ ]:
icd.loc[:,'AgeAtTest']=icd.AgeAtTest.apply(lambda x: round(x,1))
cpt.loc[:,'AgeAtTest']=cpt.AgeAtTest.apply(lambda x: round(x,1))
drug.loc[:,'AgeAtTest']=drug.AgeAtTest.apply(lambda x: round(x,1))
dexa_pd.loc[:,'AgeAtTest']=dexa_pd.AgeAtTest.apply(lambda x: round(x,1))

In [ ]:
icd.drop('AGEATCLAIM', axis=1, inplace=True)
cpt.drop('AGEATCLAIM', axis=1, inplace=True)
drug.drop('AGEATCLAIM', axis=1, inplace=True)
drug.drop('NZ_INSERT_DT', axis=1, inplace=True)
drug.drop('DURATION', axis=1, inplace=True)
adms_pd.drop('ADM_ENC_ID', axis=1, inplace=True)
dexa_pd.drop('DOB', axis=1, inplace=True)

### Load different general-equivalence mappings (GEMs)
For cross reference between development names and manuscript names:
+ CW_gems         =  Forward GEM (mapping ICD-9 diagnoses to ICD-10 diagnoses) <br>
+ CW_reimburse    =  Reverse GEM (mapping ICD-10 diagnoses to ICD-9 diagnoses) <br>

("bestmapx" were initially thought to be distinct maps but further inspection revealed they are the CW_reimburse map with ~10 links different)

In [ ]:
# import the GEMs
gems=pd.read_pickle('./pickle/gems.p')

#print('Preview of gems:\n')
#prDF(gems)

#Load the 'bestMap' (local specific name for the GEM) equ-map
#bestMap=pd.read_pickle('./pickle/bestMap.p')
#print('Preview of bestMap_pickle:\n')
#prDF(bestMap)

#Load the reimburse equ-map
#reimburse=pd.read_pickle('./pickle/reimburse.p')
#print('Preview of reimburse_pickle')
#prDF(reimburse)


In [ ]:
# Preprocess to ICDX_CM_only GEMs (Remove procedural codes, Remove expired maps, Return processed maps)
# simplify_xgems(reference mapping, rootICD that will be set, 'd'iagnosis/'p'rocedural code)
"""
CW_gems={}
CW_bestMap9={}
CW_bestMap10={}
CW_reimburse={}

# Base case
CW_gems['gems_s'] = simplify_xgem(gems, 9, 'd')
CW_bestMap9['bestMap9_s']  = simplify_xgem(bestMap, 9, 'd')
CW_bestMap10['bestMap10_s'] = simplify_xgem(bestMap, 10, 'd')
CW_reimburse['reimburse_s'] = simplify_xgem(reimburse, 10, 'd')

"""

(Irrelevant -- no ICD procedural codes were in actual PHI) Create a gems map for ICD9 to 10 procedure codes:

In [ ]:
CW_gems_p={}
CW_gems_p['gems_ps']=simplify_xgem(gems, 9, 'p')

###Inspect parsed Maps

In [ ]:
# Print current keys of GEM dictionaries:
#for x in [CW_gems, CW_bestMap9, CW_bestMap10, CW_reimburse]:
#    print(x.keys())
    #print(x.values())

### Pool the GEMs into single variable

In [ ]:
## This chunk is hashed out becaused the pooled dicts were later stored preprocessed and imported by following chunk

#pooleddict={}
#for x in [CW_gems, CW_bestMap9, CW_bestMap10, CW_reimburse]:
#    pooleddict.update(x)
# pooleddict_frompicklejar = pooleddict


In [ ]:
pooleddict_frompicklejar = pd.read_pickle('./pickle/pickled_dicts2016_11_15.p')
pooleddict_frompicklejar.keys()

Merge HCC_9 and CCS_9 into GEMs

In [ ]:
pooleddict_frompicklejar['bestMap10_s']=pooleddict_frompicklejar['bestMap10_s'].merge(hcc_9[['subcode','hcclev9']], left_on='ICD9_subcode', right_on='subcode', how='left')
pooleddict_frompicklejar['bestMap10_s']=pooleddict_frompicklejar['bestMap10_s'].merge(ccs_9[['subcode','ccslev9']], left_on='ICD9_subcode', right_on='subcode', how='left')

pooleddict_frompicklejar['bestMap9_s']=pooleddict_frompicklejar['bestMap9_s'].merge(hcc_9[['subcode','hcclev9']], left_on='ICD9_subcode', right_on='subcode', how='left')
pooleddict_frompicklejar['bestMap9_s']=pooleddict_frompicklejar['bestMap9_s'].merge(ccs_9[['subcode','ccslev9']], left_on='ICD9_subcode', right_on='subcode', how='left')

pooleddict_frompicklejar['gems_s']=pooleddict_frompicklejar['gems_s'].merge(hcc_9[['subcode','hcclev9']], left_on='ICD9_subcode', right_on='subcode', how='left')
pooleddict_frompicklejar['gems_s']=pooleddict_frompicklejar['gems_s'].merge(ccs_9[['subcode','ccslev9']], left_on='ICD9_subcode', right_on='subcode', how='left')

pooleddict_frompicklejar['reimburse_s']=pooleddict_frompicklejar['reimburse_s'].merge(hcc_9[['subcode','hcclev9']], left_on='ICD9_subcode', right_on='subcode', how='left')
pooleddict_frompicklejar['reimburse_s']=pooleddict_frompicklejar['reimburse_s'].merge(ccs_9[['subcode','ccslev9']], left_on='ICD9_subcode', right_on='subcode', how='left')

Merge HCC_10 and CCS_10 into GEMs by Left Joins

In [ ]:
pooleddict_frompicklejar['bestMap10_s']=pooleddict_frompicklejar['bestMap10_s'].merge(hcc_10[['subcode','hcclev10']], left_on='ICD10_subcode', right_on='subcode', how='left')
pooleddict_frompicklejar['bestMap10_s']=pooleddict_frompicklejar['bestMap10_s'].merge(ccs_10[['subcode','ccslev10']], left_on='ICD10_subcode', right_on='subcode', how='left')

pooleddict_frompicklejar['bestMap9_s']=pooleddict_frompicklejar['bestMap9_s'].merge(hcc_10[['subcode','hcclev10']], left_on='ICD10_subcode', right_on='subcode', how='left')
pooleddict_frompicklejar['bestMap9_s']=pooleddict_frompicklejar['bestMap9_s'].merge(ccs_10[['subcode','ccslev10']], left_on='ICD10_subcode', right_on='subcode', how='left')

pooleddict_frompicklejar['gems_s']=pooleddict_frompicklejar['gems_s'].merge(hcc_10[['subcode','hcclev10']], left_on='ICD10_subcode', right_on='subcode', how='left')
pooleddict_frompicklejar['gems_s']=pooleddict_frompicklejar['gems_s'].merge(ccs_10[['subcode','ccslev10']], left_on='ICD10_subcode', right_on='subcode', how='left')

pooleddict_frompicklejar['reimburse_s']=pooleddict_frompicklejar['reimburse_s'].merge(hcc_10[['subcode','hcclev10']], left_on='ICD10_subcode', right_on='subcode', how='left')
pooleddict_frompicklejar['reimburse_s']=pooleddict_frompicklejar['reimburse_s'].merge(ccs_10[['subcode','ccslev10']], left_on='ICD10_subcode', right_on='subcode', how='left')

For neatness, drop unnecessary columns from GEMs

In [ ]:
for x,y in pooleddict_frompicklejar.items():
    y.drop(['subcode_x' , 'subcode_y'], axis=1, inplace=True)
    try:
        y.drop([u'ICD10_EFFECTIVE_DATE', u'ICD10_EXPIRATION_DATE',
                u'ICD9_EFFECTIVE_DATE', u'ICD9_EXPIRATION_DATE',
              u'MAPPING_EFFECTIVE_DATE' ,u'MAPPING_EXPIRATION_DATE'], axis=1, inplace=True)
    
    except:
        y.drop(['ICD9_EFFECTIVE_TIMESTAMP', 'ICD9_EXPIRATION_TIMESTAMP',
                'ICD10_EFFECTIVE_TIMESTAMP', 'ICD10_EXPIRATION_TIMESTAMP',
                'MAPPING_EFFECTIVE_TIMESTAMP', 'MAPPING_EXPIRATION_TIMESTAMP', 'NZ_INSERT_DT'],
               axis=1, inplace=True)
    print(y.columns.values)

In [ ]:
for x,y in pooleddict_frompicklejar.items():
    y['ICD10_subcode']=y['ICD10_subcode'].apply(lambda x: str(x).replace('x','X'))
print('truthfully, this should be irrelevant, as Vanderbilt is not using any ICD10 codes that contain an X')

### Some data checks:

In [ ]:
# Proofing preprocessing before going forward
verification1 = pooleddict_frompicklejar['gems_s'][['ICD10_CODE','ICD10_subcode','ICD9_CODE']]
verification1['first']=verification1.ICD10_subcode.str[0]
verification1['end']= verification1.ICD10_subcode.str[1:]
verification1=verification1[verification1['end'].str.contains('X')]
verification1['trunc']= verification1['ICD10_subcode'].apply(lambda x:str(x)[1:].split('X')[0])
verification1['trunc']=verification1['first']+verification1['trunc']
verification1['length']=verification1['trunc'].apply(lambda x: len(str(x)))
lx= verification1.groupby('length').count()
lx.reset_index(inplace=True)
lxmn=min(lx.length)
lxmx=max(lx.length)
print('{}-{}'.format(lxmn,lxmx))
tester = icd[['MRN','CODE','EVT_DATE']]
for x in range(lxmn, lxmx+1):
    tester[str(x)]=tester['CODE'].str[0:x]

for x in range(lxmn, lxmx+1):
    if len(tester[tester[str(x)].isin(verification1.trunc)])>0:
        raise ValueError('algorithm not tested for ICD10 codes that contain wildcard-Xs')
    else:
        print('pass')
del verification1
del tester

In [ ]:
### Quality Assurance: Are there any ICD Procedure Code in claims data?
if len(icd[icd['CODE'].apply(lambda x: re.search('^\w{7}', x) !=None, re.I)]) >0:
    raise ValueError('ICD10 claims data include ICD10 Procedure Codes - which are not supported by current script')
if len(icd[icd['ICD9_CODE'].apply(lambda x: re.search('^\w{2}[.]', x) !=None, re.I)]) >0:
    raise ValueError('ICD9 claims data include ICD9 Procedure Codes - which are not supported by current script')

# Load the classes for each metric

### Instantiate a class for each of the 11 metrics

In [ ]:
#Set the defaults
MetricClass.load_dataframes(c_servdata=cpt, c_claimdata=icd, 
                            c_demodata=demo, c_demoref=democriteria,
                            c_indicref=indications, c_redflagref=redflags,
                            c_servref=services, c_dexadata=None,
                            c_preopreftab=None, c_admsdata=None,
                            c_newindicref=indic_10, c_newredflagref=red_10,
                            c_newservref=serv_10, c_window_date= demoStartDate)
MetricClass.load_gemdictionaries(pooleddict_frompicklejar, CW_gems_p)

In [ ]:
# Add the standard alternate arguments:
MetricClass.appendservdata_dict('drug',drug)
MetricClass.appenddexadata_dict('dexadata',dexa_pd)
MetricClass.appendadmsdata_dict('admsdata',adms_pd)
MetricClass.appendpreopreftab_dict('preopref',preopref)

In [ ]:
# Metric Classes:
# [arguments]             
CW_cerv     = MetricClass('cerv', lookback=365) #ideally, 3650 days
CW_card     = MetricClass('card')
CW_vitd     = MetricClass('vitd')
CW_bph      = MetricClass('bph',  tempsort_day=60)
CW_lbp      = MetricClass('lbp',  tempsort_day=42)
CW_feed     = MetricClass('feed', tempsort_day = False)
CW_psyc     = MetricClass('psyc', servdata= 'drug', second_datax='SERVDATA') 
CW_dexa     = MetricClass('dexa', dexadata='dexadata', tempsort_day=730, lookback=365) #ideally, these are both 730 days, but at time, only 365 days of diagnosis data available in ICD-10 era
CW_narc     = MetricClass('narc', servdata='drug', admsdata='admsdata', preopreftab='preopref', tempsort_day=60, second_datax='SERVDATA') #ideally, tempsort_day=60
CW_nonpreop = MetricClass('nonpreop', admsdata='admsdata', preopreftab='preopref', tempsort_day=30)
CW_catpreop = MetricClass('catpreop', admsdata='admsdata', preopreftab='preopref', tempsort_day=30)


### Preparing a report on 11 metrics in single script

In [ ]:
# Metrics
elevenmetrics= [('CW_feed',CW_feed),('CW_narc',CW_narc),('CW_card',CW_card),
                ('CW_psyc',CW_psyc), ('CW_nonpreop',CW_nonpreop),
                ('CW_catpreop',CW_catpreop),
                ('CW_lbp',CW_lbp),('CW_dexa',CW_dexa),
                ('CW_cerv',CW_cerv),('CW_vitd',CW_vitd),
                ('CW_bph',CW_bph)]


# fewmetrics = [('CW_card',CW_card)] -- used for debugging


In [ ]:
os.chdir('./exported')

In [ ]:
""" # For examining single metric, used for debugging
from cw_package import chooseallwisely
testdict=pooleddict_frompicklejar['gems_s']

j=chooseallwisely('narc',                         #0
                    servdata=drug,                       #1
                    claimdata=icd,                      #2
                    demodata=demo,                      #3
                    demoref=democriteria,               #4
                    indicref=indications,               #5
                    redflagref=redflags,                #6
                    servref=services,                   #7
                    dexadata=None,                      #8
                    preopreftab=preopref,                   #9
                    admsdata=adms_pd,                     #10
                    tempsort_day=60,                 #11
                    lookback=365,                      #12
                    rootICD_gx=9,                     #13
                    sourcegemdict=testdict,            #14
                    procgem_dict=None,              #15
                    uniqueflagx=False,               #16
                    second_data=cpt,                 #17
                    window_date = '2016-04-01'              #18
                    )
"""

#### Run the mapping of reference codes from ICD-9 to ICD-10, and calculate all 11 metrics on patient data in ICD-10

In [ ]:
f_refto10_gems_dict={}
for x in elevenmetrics:
    try:
        y=x[1].run_refto10_gems()
        y['denominator']= y['denominator'][y['denominator'].TEST_DATE_a>=demoStartDate]
        y['denominator']= y['denominator'][y['denominator'].TEST_DATE_a<demoEndDate]
        y['numerator']= y['numerator'][y['numerator'].TEST_DATE_a>=demoStartDate]
        y['numerator']= y['numerator'][y['numerator'].TEST_DATE_a<demoEndDate]
        f_refto10_gems_dict[x[0]]={'denominator_l':len(y['denominator']),'denominator':y['denominator'],
                        'numerator_l':len(y['numerator']),'numerator':y['numerator']}
    except:
        y={}
        y['denominator']='not calculated'
        y['numerator']  ='not calculated'
        f_refto10_gems_dict[x[0]]={'denominator_l':'na','denominator':y['denominator'],
                        'numerator_l':'na','numerator':y['numerator']}
# refto10_gems

#### Run the mapping of patient data from ICD-10 to ICD-9, and calculate all 11 metrics in ICD-9 against reference ICD-9 definitions

In [ ]:
#for k,v in final_dict.iteritems():
#    print(final_dict[k]['metric_name'])

f_claimto9_reimb_dict={}
for x in elevenmetrics:
    try:
        y=x[1].run_claimto9_reimb()
        y['denominator']= y['denominator'][y['denominator'].TEST_DATE_a>=demoStartDate]
        y['denominator']= y['denominator'][y['denominator'].TEST_DATE_a<demoEndDate]
        y['numerator']= y['numerator'][y['numerator'].TEST_DATE_a>=demoStartDate]
        y['numerator']= y['numerator'][y['numerator'].TEST_DATE_a<demoEndDate]
        f_claimto9_reimb_dict[x[0]]={'denominator_l':len(y['denominator']),'denominator':y['denominator'],
                        'numerator_l':len(y['numerator']),'numerator':y['numerator']}
    except:
        y={}
        y['denominator']='not calculated'
        y['numerator']  ='not calculated'
        f_claimto9_reimb_dict[x[0]]={'denominator_l':'na','denominator':y['denominator'],
                        'numerator_l':'na','numerator':y['numerator']}



# claimto9_reimb

#### Calculate the 11 metrics by using the Novel ICD-10 based reference codes against the patient data in ICD-10. 
No actual mapping across ICD generations is done for this metric.

In [ ]:
f_10to10_dict={}
for x in elevenmetrics:
    y=x[1].run_10to10()
    y['denominator']= y['denominator'][y['denominator'].TEST_DATE_a>=demoStartDate]
    y['denominator']= y['denominator'][y['denominator'].TEST_DATE_a<demoEndDate]
    y['numerator']= y['numerator'][y['numerator'].TEST_DATE_a>=demoStartDate]
    y['numerator']= y['numerator'][y['numerator'].TEST_DATE_a<demoEndDate]
    f_10to10_dict[x[0]]={'denominator_l':len(y['denominator']),'denominator':y['denominator'],
                    'numerator_l':len(y['numerator']),'numerator':y['numerator']}

In [ ]:
### this invokes an additional GEM that on inspection was the same as the reimbursement GEM implemented above (see comment above where GEMs were loaded)
f_refto10_bestMap9_dict={}
for x in elevenmetrics:
    try:    
        y=x[1].run_refto10_bestMap9()
        y['denominator']= y['denominator'][y['denominator'].TEST_DATE_a>=demoStartDate]
        y['denominator']= y['denominator'][y['denominator'].TEST_DATE_a<demoEndDate]
        y['numerator']= y['numerator'][y['numerator'].TEST_DATE_a>=demoStartDate]
        y['numerator']= y['numerator'][y['numerator'].TEST_DATE_a<demoEndDate]
        f_refto10_bestMap9_dict[x[0]]={'denominator_l':len(y['denominator']),'denominator':y['denominator'],
                        'numerator_l':len(y['numerator']),'numerator':y['numerator']}
    except:
        y={}
        y['denominator']='not calculated'
        y['numerator']  ='not calculated'
        f_refto10_bestMap9_dict[x[0]]={'denominator_l':'na','denominator':y['denominator'],
                        'numerator_l':'na','numerator':y['numerator']}
# refto10_bestMap

In [ ]:
### This invokes an additional GEM that on inspection was the same as the reimbursement GEM above (see comment above where GEMs were loaded)

f_claimto9_bestMap10_dict={}
for x in elevenmetrics:
    try:
        y=x[1].run_claimto9_bestMap10()
        y['denominator']= y['denominator'][y['denominator'].TEST_DATE_a>=demoStartDate]
        y['denominator']= y['denominator'][y['denominator'].TEST_DATE_a<demoEndDate]
        y['numerator']= y['numerator'][y['numerator'].TEST_DATE_a>=demoStartDate]
        y['numerator']= y['numerator'][y['numerator'].TEST_DATE_a<demoEndDate]
        f_claimto9_bestMap10_dict[x[0]]={'denominator_l':len(y['denominator']),'denominator':y['denominator'],
                        'numerator_l':len(y['numerator']),'numerator':y['numerator']}
    except:
        y={}
        y['denominator']='not calculated'
        y['numerator']  ='not calculated'
        f_claimto9_bestMap10_dict[x[0]]={'denominator_l':'na','denominator':y['denominator'],
                        'numerator_l':'na','numerator':y['numerator']}

# claimto9_bestMap10

Store output for later access

In [ ]:
f= open(coded_date+'/pickled_f_10to10_'+coded_date+'.p','w')
pickle.dump(f_10to10_dict, f)
f.close()

#del f_10to10_dict

In [ ]:
f= open(coded_date+'/pickled_f_refto10_gems_'+coded_date+'.p','w')
pickle.dump(f_refto10_gems_dict, f)
f.close()

#del f_refto10_gems_dict

In [ ]:
f= open(coded_date+'/pickled_f_claimto9_reimb_'+coded_date+'.p','w')
pickle.dump(f_claimto9_reimb_dict, f)
f.close()

#del f_claimto9_reimb_dict

In [ ]:
f= open(coded_date+'/pickled_f_refto10_bestMap9_'+coded_date+'.p','w')
pickle.dump(f_refto10_bestMap9_dict, f)
f.close()

#del f_refto10_bestMap9_dict

In [ ]:
f= open(coded_date+'/pickled_f_claimto9_bestMap10_'+coded_date+'.p','w')
pickle.dump(f_claimto9_bestMap10_dict, f)
f.close()


Everything below is for operational reporting. The preceding pickled files are used by subsequent Step_X notebooks for the GEM-study.

--- BREAK ----

### Arrange operational reports for numerators and denominators across the 11 metrics, by GEM implementation

In [ ]:
#claimto9_bestMap10_dict = pd.read_pickle('./pickle/pickled_f_claimto9_bestMap10_2016_08_29.p')
#f_claimto9_reimb_dict= pd.read_pickle('./'+coded_date+'/pickled_f_claimto9_reimb_'+coded_date+'.p')
#f_refto10_gems_dict= pd.read_pickle('./'+coded_date+'/pickled_f_refto10_gems_'+coded_date+'.p')
#f_refto10_bestMap9_dict = pd.read_pickle('./'+coded_date+'/pickled_f_refto10_bestMap9_'+coded_date+'.p')
#f_10to10_dict = pd.read_pickle('./'+coded_date+'/pickled_f_10to10_'+coded_date+'.p')

ten_to_10 = ('10to10', f_10to10_dict)
claimto9_reimb = ('claimto9_reimb', f_claimto9_reimb_dict)
refto10_gems = ('refto10_gems',f_refto10_gems_dict)
refto10_bestMap9 = ('refto10_bestMap9',f_refto10_bestMap9_dict)
claimto9_bestMap10 = ('claimto9_bestMap10',f_claimto9_bestMap10_dict)

collected = [ten_to_10, claimto9_reimb, refto10_gems, refto10_bestMap9, claimto9_bestMap10]

In [ ]:
tallied_ratios= {}
for x in collected:
    tallied_ratios[x[0]]={y:(v['numerator_l'],v['denominator_l']) for y,v in x[1].iteritems()}

tallied_ratios.keys()
#import pprint

In [ ]:
summreport = tallied_ratios
f= open(coded_date+'/pickled_tallied_ratios_'+coded_date+'.p','w')
pickle.dump(tallied_ratios, f)
f.close()

In [ ]:
# this was necessary when ICD procedural codes were possible -- but this was not an issue in final version
for x in collected: 
    try:
        if x[1]['CW_feed']['denominator_l']=='na':
            del x[1]['CW_feed'] 
            print('deletion committed')
    except:
        print('nothing')
        pass
        

In [ ]:
# Monthly report of low-value services, exported to CSV
def output_monthly(cwreport, filename,suffix):
    try:
        z=pd.concat({k:pd.concat([y['numerator'].groupby('TEST_DATE_month')['MRN'].apply(lambda x:len(np.unique(x))),
                   y['denominator'].groupby('TEST_DATE_month')['MRN'].apply(lambda x:len(np.unique(x)))],axis=1) for k,y in cwreport.iteritems()},axis=0)
        z.columns=["Numerator","Denominator"]
        z['Proportion']=z['Numerator'].div(z['Denominator'])
        z=z.reset_index()
        z.rename(columns={'level_0':'Long_Metric','Numerator':'Performance Not Met','Denominator':'All Performance'},inplace=True)
        z=z[z.TEST_DATE_month.isin(reportOnMonths)].fillna(0)
        z['Metric']=z.Long_Metric.replace("^.._","", regex=True)
        z=z.merge(democriteria,left_on='Metric',right_on='key',how='left')
        reportFilename=filename
        z.fillna(0).to_csv(coded_date+'/'+reportFilename+suffix+'.csv')
    except:
        raise ValueError("Check to see that there are no \'na\' string values in x[1]")


In [ ]:
from pprint import pprint
pprint(summreport)

In [ ]:
keylist = ['CW_cerv', 'CW_nonpreop', 'CW_bph', 'CW_card', 'CW_vitd',  'CW_lbp',  'CW_narc',  'CW_dexa', 'CW_catpreop',  'CW_psyc', 'CW_feed']

In [ ]:
# get the individual patient id's for inspection
def extract_MRNs(dictionary, keylistx):
    MRNs_num = {}
    MRNs_denom = {}
    x = dictionary
    for y in keylistx:
        try:
            x[y]['numerator']['metric']=y
            x[y]['denominator']['metric']=y            
            MRNs_num[y]=x[y]['numerator'][['MRN','TEST_DATE_month']]
            MRNs_denom[y]=x[y]['denominator'][['MRN','TEST_DATE_month']]
            print('success')
        except:
            try:
                x[y]['denominator']['metric']=y            
                MRNs_denom[y]=x[y]['denominator'][['MRN','TEST_DATE_month']]
            except:
                pass
    return (MRNs_num, MRNs_denom)
    

In [ ]:
MRN_10to10 =           ('10to10', extract_MRNs(ten_to_10[1], keylist))
MRNclaimto9_reimb =    ('claimto9_reimb',extract_MRNs(claimto9_reimb[1], keylist))
MRNrefto10_gems=       ('refto10_gems',extract_MRNs(refto10_gems[1], keylist))
MRNrefto10_bestMap9=   ('refto10_bestMap9',extract_MRNs(refto10_bestMap9[1], keylist))
MRNclaimto9_bestMap10= ('claimto9_bestMap10',extract_MRNs(claimto9_bestMap10[1], keylist))

collectedMRNs= [MRN_10to10, MRNclaimto9_reimb, MRNrefto10_gems, MRNrefto10_bestMap9, MRNclaimto9_bestMap10]

In [ ]:
# export MRNs on monthly basis
def output_monthlyMRNs(cwreport, filename,suffix, term):
    alpha=0 if term=='numerator' else 1
    try:
        z=pd.concat({k:pd.concat([y],axis=1) for k,y in cwreport[alpha].iteritems()},axis=0)
        #z.columns=["Numerator","Denominator"]
        z=z.reset_index()
        z.rename(columns={'level_0':'Long_Metric'},inplace=True)
        z=z.drop(['level_1'],axis=1)
        z=z[z.TEST_DATE_month.isin(reportOnMonths)].fillna(0)
        z['Metric']=z.Long_Metric.replace("^.._","", regex=True)
        z=z.merge(democriteria,left_on='Metric',right_on='key',how='left')
        reportFilename=filename
        z.fillna(0).to_csv(coded_date+'/'+reportFilename+term+suffix+'.csv')
    except:
        raise ValueError("Check to see that there are no \'na\' string values in x[1]")

In [ ]:
#j=collected[3][1]
#for x,y in j.iteritems():
#    print(y['denominator'].columns.values)

In [ ]:
for x in collectedMRNs:
    print('progress')
    output_monthlyMRNs(x[1],x[0],coded_date, 'numerator')
    output_monthlyMRNs(x[1],x[0],coded_date, 'denominator')
    print('cycle complete')
    